In [ ]:
# !pip install numpy==1.20.3
# !pip install pyheif==0.5.1
# !pip install requests==2.26.0
# !pip install clip-by-openai
# !pip install matplotlib==3.4.2
# !pip install torch==1.7.1
# !pip install whatimage==0.0.3
# !pip install streamlit==0.89.0
# !pip install Pillow==8.3.2
# !pip install opencv-python==4.5.3.56
# !pip install pyaml
# !pip install jupyter


In [1]:
import cv2
from glob import glob
from pathlib import Path
import smtplib

from model import Model
from utils import plot
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
model = Model()

In [ ]:
image_pathes = glob('./data/*.jpg')
image_pathes += glob('./data/*.png')
image_pathes[:2]

In [6]:
save_folder_path = Path('results')

In [ ]:
for i, image_path in enumerate(image_pathes):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label = model.predict(image)['label']
    plot(image=image, title=label, 
         save_path=save_folder_path / f'{i}.jpg')

# Read video 
### and save predictions for each frame as another video

In [3]:
def process_video(input_video_path: str, output_path: str):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
    size = (width, height)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(cap.get(cv2.CAP_PROP_FPS) + 0.5)
    print('FPS', fps)
    success, frame = cap.read()
    
    out = cv2.VideoWriter(save_video_path, fourcc, fps, size)
    success = True


    while success and cap.isOpened():

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        prediction = model.predict(frame)
        label = prediction['label']
        conf = prediction['confidence']
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.putText(frame, label.title(), 
                            (0, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, 
                            (0, 0, 255), 2)

        cv2.imshow('Recording...', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        out.write(frame)
        success, frame = cap.read()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [7]:
video_path = './data/oki2.mp4'
save_video_path = str(save_folder_path / 'k.avi')

process_video(video_path, save_video_path) 

FPS 60


KeyboardInterrupt: 

In [ ]:
video_path = './data/office_fight.mp4'
save_video_path = str(save_folder_path / 'result_fight.avi')

process_video(video_path, save_video_path)

In [ ]:
video_path = './data/test1.mp4'
save_video_path = str(save_folder_path / 'test1.avi')

process_video(video_path, save_video_path)

In [ ]:
import cv2
from model import Model
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

model = Model()

def send_email(subject, body):
    sender_email = 'ilyassbennane123@gmail.com'  # Sender's email address
    receiver_email = 'cherkaniaymen4@gmail.com'  # Recipient's email address
    password = 'zivt mvam qory gowq'  # Sender's email password
    # Create message container
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach body to the email
    msg.attach(MIMEText(body, 'plain'))

    # Connect to SMTP server and send email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)
        print("Email sent successfully")
    except Exception as e:
        print("Error sending email:", str(e))
    finally:
        if 'server' in locals():
            server.quit()

def process_realtime():
    # Open the default camera
    cap = cv2.VideoCapture(0)  # Use 0 for the default camera, or change it to another index if you have multiple cameras

    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame")
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Predict violence
        prediction = model.predict(frame_rgb)
        label = prediction['label']
        confidence = prediction['confidence']

        # Display the result
        frame = cv2.putText(frame, f"{label}: {confidence:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # If violence is detected, send an email
        if label == 'violence in office':
            send_email("Violence Detected", "Violence has been detected in the video stream.")

        # Display the resulting frame
        cv2.imshow('Real-time Violence Detection', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release everything when done
    cap.release()
    cv2.destroyAllWindows()


: 

In [ ]:
process_realtime()


In [ ]:
video_path = './data/test7.mp4'
save_video_path = str(save_folder_path / 'test7.avi')

process_video(video_path, save_video_path)